***
<font size="6"><center><b> HD-CapsNet: A Hierarchical Deep Capsule Network for Image Classification </b></center></font>
***

**Changing Model Architecture**
- **(Mod-3.1)** 32D>16D>8D (Coarse>Medium>FINE) use skip connections between Secondary Capsules $Concatenate([P_{caps}, S_{coarse}])$ > input for $S_{medium}$ and $Concatenate([P_{caps}, S_{medium}])$ > input for $S_{fine}$
- With $L_{Cons}$

# Files and Libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras import backend as K

import tensorflow_docs as tfdocs
import tensorflow_docs.plots

# Supporting Libraries:
    #Mathplot lib for ploting graphs
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
    # numpy and pandas
import numpy as np
import pandas as pd
    #system
import os
import sys
import csv
    #import other libraries
import math
import random
from datetime import datetime
from treelib import Tree
    # ML model, Dataset and evalution metrics
sys.path.append('../../') ### adding system parth for src folder
from src import datasets # load datasets
from src import MixUp_add_loss # load datasets
from src import metrics # load hierarchcial metrics
from src import sysenv # load hierarchcial metrics
from src import models # load machine learning models

    ## Tensorflow_docs
import tensorflow_docs as tfdocs
import tensorflow_docs.plots

    # Auto reload local libraries if updated
%load_ext autoreload
%autoreload 2

# System information & GPU growth

In [2]:
systeminfo = sysenv.systeminfo()
print(systeminfo)


• Computer Name = 30BFPC1AXW95
• Working Directory = C:\Users\knoor\OneDrive - Deakin University\Deep Learning with Python\Google_Drive\Projects\Deep Learning\HD-CapsNet\Training_and_Analysis\3_CIFAR-10
• Python Version = 3.8.12 (default, Oct 12 2021, 03:01:40) [MSC v.1916 64 bit (AMD64)]
• TensorFlow Version = 2.8.0
• Keras Version = 2.8.0
• Current Environment = Anaconda Environment Name : py38tf


In [3]:
gpus = "0,1,2,3,4,5,6,7" ## Selecting Available gpus (Multi-GPUS)
gpus = "0" ## Selecting Available gpus (Single GPU)
gpugrowth = sysenv.gpugrowth(gpus = gpus) ## Limiting GPUS from OS environment
gpugrowth.memory_growth() #GPU memory growth

Following GPUS are selected =  0
Done: GPU PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
1 Physical GPUs, 1 Logical GPUs


## log directory

In [4]:
directory = sysenv.log_dir('3_CIFAR_10/HD_CapsNet/Mod_4_1_2')

# Import Dataset : CIFAR-10

In [5]:
dataset = datasets.CIFAR10(version = 'ALL') # importing CIFAR10 Dataset
# dataset = datasets.CIFAR10(version = 'reduce') # importing CIFAR10 Dataset

CIFAR-10 dataset: Training have 50,000 samples and testing have 10,000 samples


# Parameters

In [6]:
train_params = {"n_epochs" : 100,
                "batch_size": 64,
                "lr": 0.001, # Initial learning rate
                "lr_decay": 0.95, # Learning rate decay
                "decay_exe": 9, #learning rate decay execution epoch after
               }
model_params = {"optimizer": tf.keras.optimizers.Adam(train_params['lr']),
                "loss_function": models.MarginLoss(),
               }

## Learning Rate Decay Scheduler

In [7]:
def scheduler(epoch):
    learning_rate_init = train_params["lr"]
    
    if epoch > train_params["decay_exe"]:
        learning_rate_init = train_params["lr"] * (train_params["lr_decay"] ** (epoch-9))
        
    tf.summary.scalar('learning rate', data=learning_rate_init, step=epoch)
        
    return learning_rate_init

# Bottom up Approach

## Loss Function

In [8]:
number_of_classes_c = len(np.unique(np.argmax(dataset['y_train_coarse'], axis=1)))
number_of_classes_m = len(np.unique(np.argmax(dataset['y_train_medium'], axis=1)))
number_of_classes_f = len(np.unique(np.argmax(dataset['y_train_fine'], axis=1)))

## For Dynamic LossWeights
initial_lw = models.initial_lw({"coarse": number_of_classes_c,
                                "medium": number_of_classes_m,
                                "fine": number_of_classes_f})

lossweight = {'coarse_lw' : K.variable(value = initial_lw['coarse'], dtype="float32", name="coarse_lw"),
             'medium_lw' : K.variable(value = initial_lw['medium'], dtype="float32", name="medium_lw"),
             'fine_lw' : K.variable(value = initial_lw['fine'], dtype="float32", name="fine_lw"),
              'decoder_lw' : 0.1
             }

In [9]:
lossweight

{'coarse_lw': <tf.Variable 'coarse_lw:0' shape=() dtype=float32, numpy=0.4473684>,
 'medium_lw': <tf.Variable 'medium_lw:0' shape=() dtype=float32, numpy=0.31578946>,
 'fine_lw': <tf.Variable 'fine_lw:0' shape=() dtype=float32, numpy=0.23684211>,
 'decoder_lw': 0.1}

In [10]:
def margin_loss(y_true, y_proba):
    y_proba = tf.math.divide(y_proba,tf.reshape(tf.reduce_sum(y_proba,-1),(-1,1),name='reshape'),name='Normalising_Probability')
    present_error_raw = tf.square(tf.maximum(0., 0.9 - y_proba), name="present_error_raw")
    absent_error_raw = tf.square(tf.maximum(0., y_proba - 0.1), name="absent_error_raw")
    L = tf.add(y_true * present_error_raw, 0.5 * (1.0 - y_true) * absent_error_raw,name="L")
    total_marginloss = tf.reduce_sum(L, axis=1, name="margin_loss")

    return total_marginloss

def CustomLoss(y_true_c, y_true_m, y_true_f, y_pred_c, y_pred_m, y_pred_f, LW_C, LW_M, LW_F,
               C_Weight=lossweight['decoder_lw']):
    # getting the loss for each level
    loss_coarse = margin_loss(y_true_c, y_pred_c)
    loss_medium = margin_loss(y_true_m, y_pred_m)
    loss_fine = margin_loss(y_true_f, y_pred_f)

    # Check for consistency between levels using Bayes' theorem
    # Convert predicted probabilities to log probabilities for numerical stability
    # coarse_predictions = 1.0-tf.reduce_sum(y_true_c*y_pred_c, axis=1)
    # medium_predictions = 1.0-tf.reduce_sum(y_true_m*y_pred_m, axis=1)
    # fine_predictions = 1.0-tf.reduce_sum(y_true_f*y_pred_f, axis=1)
    prob_coarse = 1.0-loss_coarse
    prob_medium = 1.0-loss_medium
    prob_fine = 1.0-loss_fine
    
    log_pred_coarse = tf.math.log(tf.clip_by_value(prob_coarse, 1e-10, 1.0))
    log_pred_medium = tf.math.log(tf.clip_by_value(prob_medium, 1e-10, 1.0))
    log_pred_fine = tf.math.log(tf.clip_by_value(prob_fine, 1e-10, 1.0))

    prob_fine_to_medium = tf.reduce_sum(-(log_pred_fine-log_pred_medium))
    prob_medium_to_coarse = tf.reduce_sum(-(log_pred_medium-log_pred_coarse))
    
    batch_loss = LW_C*loss_coarse + LW_M*loss_medium + LW_F*loss_fine + C_Weight*(prob_fine_to_medium+prob_medium_to_coarse)

    return tf.reduce_mean(batch_loss)

## Model Architecture

In [13]:
def get_compiled_model():
    input_shape = dataset['x_train'].shape[1:]

    input_shape_yc = dataset['y_train_coarse'].shape[1:]
    input_shape_ym = dataset['y_train_medium'].shape[1:]
    input_shape_yf = dataset['y_train_fine'].shape[1:]

    no_coarse_class = 2
    no_medium_class = 7
    no_fine_class = 10

    PCap_n_dims = 8

    SCap_f_dims = 8
    SCap_m_dims = 16
    SCap_c_dims = 32


    # Input image
    x_input = keras.layers.Input(shape=input_shape, name="Input_Image")

    # Input True Labels
    y_c = keras.layers.Input(shape=input_shape_yc, name='input_yc')
    y_m = keras.layers.Input(shape=input_shape_ym, name='input_ym')
    y_f = keras.layers.Input(shape=input_shape_yf, name='input_yf')

    #--- block 1 ---
    x = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(x_input)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    #--- block 2 ---
    x = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    #--- block 3 ---
    x = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    #--- block 4 ---
    x = keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)


    # Layer 3: Reshape to 8D primary capsules 
    reshapec = keras.layers.Reshape((int((tf.reduce_prod(x.shape[1:]).numpy())/PCap_n_dims), 
                                     PCap_n_dims), name="reshape_layer")(x)
    p_caps = keras.layers.Lambda(models.squash, name='p_caps')(reshapec)

    ## Layer Secondary Capsule: For coarse level
    s_caps_c = models.SecondaryCapsule(n_caps=no_coarse_class, n_dims=SCap_c_dims, 
                        name="s_caps_coarse")(p_caps)
    
    ## Skip Connection: For Medium Level
    p_caps_m = keras.layers.Reshape((int((tf.reduce_prod(p_caps.shape[1:]).numpy())/s_caps_c.shape[-1]),
                                     s_caps_c.shape[-1]), name="primary_skip_m")(p_caps)
    skip_m = keras.layers.Concatenate(axis=1)([p_caps_m, s_caps_c])

    ## Layer Secondary Capsule: For medium level
    s_caps_m = models.SecondaryCapsule(n_caps=no_medium_class, n_dims=SCap_m_dims, 
                        name="s_caps_medium")(skip_m)
    
    ## Skip Connection: For Fine Level
    p_caps_f = keras.layers.Reshape((int((tf.reduce_prod(p_caps.shape[1:]).numpy())/s_caps_m.shape[-1]),
                                     s_caps_m.shape[-1]), name="primary_skip_f")(p_caps)
    skip_f = keras.layers.Concatenate(axis=1)([p_caps_f, s_caps_m])

    ## Layer Secondary Capsule: For fine level
    s_caps_f = models.SecondaryCapsule(n_caps=no_fine_class, n_dims=SCap_f_dims, 
                        name="s_caps_fine")(skip_f)

    pred_c = models.LengthLayer(name='prediction_coarse')(s_caps_c)

    pred_m = models.LengthLayer(name='prediction_medium')(s_caps_m)

    pred_f = models.LengthLayer(name='prediction_fine')(s_caps_f)

    model = keras.Model(inputs= [x_input, y_c, y_m, y_f],
                        outputs= [pred_c, pred_m, pred_f],
                        name='HD-CapsNet')
    
    ## Saving Model Architecture
    keras.utils.plot_model(model, to_file = directory+"/Architecture.png", show_shapes=True)

    model.add_loss(CustomLoss(y_c, y_m, y_f, pred_c, pred_m, pred_f, 
                              lossweight['coarse_lw'], lossweight['medium_lw'], lossweight['fine_lw']
                              )
                    )

    model.compile(optimizer='adam',                  
                  metrics={'prediction_fine': 'accuracy',
                           'prediction_medium': 'accuracy',
                           'prediction_coarse': 'accuracy'
                          }
    )
    return model

In [14]:
model = get_compiled_model()

In [15]:
model.summary()

Model: "HD-CapsNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input_Image (InputLayer)       [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 block1_conv1 (Conv2D)          (None, 32, 32, 64)   1792        ['Input_Image[0][0]']            
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 64)  256         ['block1_conv1[0][0]']           
 alization)                                                                                       
                                                                                                  
 block1_conv2 (Conv2D)          (None, 32, 32, 64)   36928       ['batch_normalization[0]

                                                                                                  
 prediction_medium (LengthLayer  (None, 7)           0           ['s_caps_medium[0][0]']          
 )                                                                                                
                                                                                                  
 prediction_fine (LengthLayer)  (None, 10)           0           ['s_caps_fine[0][0]']            
                                                                                                  
 tf.math.reduce_sum (TFOpLambda  (None,)             0           ['prediction_coarse[0][0]']      
 )                                                                                                
                                                                                                  
 tf.math.reduce_sum_2 (TFOpLamb  (None,)             0           ['prediction_medium[0][0]']      
 da)      

                                                                                                  
 tf.math.multiply_7 (TFOpLambda  (None, 10)          0           ['tf.math.subtract_8[0][0]']     
 )                                                                                                
                                                                                                  
 tf.math.square_5 (TFOpLambda)  (None, 10)           0           ['tf.math.maximum_5[0][0]']      
                                                                                                  
 tf.math.multiply (TFOpLambda)  (None, 2)            0           ['input_yc[0][0]',               
                                                                  'tf.math.square[0][0]']         
                                                                                                  
 tf.math.multiply_2 (TFOpLambda  (None, 2)           0           ['tf.math.multiply_1[0][0]',     
 )        

 da)                                                                                              
                                                                                                  
 tf.math.reduce_sum_7 (TFOpLamb  ()                  0           ['tf.math.negative_1[0][0]']     
 da)                                                                                              
                                                                                                  
 tf.__operators__.add (TFOpLamb  (None,)             0           ['tf.math.multiply_9[0][0]',     
 da)                                                              'tf.math.multiply_10[0][0]']    
                                                                                                  
 tf.math.multiply_11 (TFOpLambd  (None,)             0           ['tf.math.reduce_sum_5[0][0]']   
 a)                                                                                               
          

## Data Augmentation

In [16]:
datagen = ImageDataGenerator(width_shift_range=0.1,
                            height_shift_range=0.1)

training_generator = MixUp_add_loss.MixupGenerator_3level(dataset['x_train'],
                                                 dataset['y_train_coarse'], 
                                                 dataset['y_train_medium'],
                                                 dataset['y_train_fine'],
                                                 batch_size=train_params["batch_size"],
                                                 alpha=0.2, 
                                                 datagen=datagen
                                                )()

## Callback

In [17]:
tb = keras.callbacks.TensorBoard(directory+'./tb_logs'+ datetime.now().strftime("%Y%m%d-%H%M%S"))
log = keras.callbacks.CSVLogger(directory+'/log.csv', append=True)

checkpoint = keras.callbacks.ModelCheckpoint(
    directory+'/epoch-best.h5', 
    monitor='val_prediction_fine_accuracy',
    save_best_only=True, save_weights_only=True, verbose=1)
change_lw = models.LossWeightsModifier(lossweight = lossweight,
                               initial_lw = initial_lw,
                               directory = directory)
lr_decay = keras.callbacks.LearningRateScheduler(scheduler)

## Model Training

In [18]:
model_save_dir = str(directory+'/trained_model.h5')
try:
    model.load_weights(model_save_dir)
except:
    history = model.fit(training_generator,
                        steps_per_epoch = int(dataset['x_train'].shape[0] / train_params["batch_size"]),
                        epochs = train_params["n_epochs"],
                        validation_data = ([dataset['x_test'],
                                            dataset['y_test_coarse'],dataset['y_test_medium'],dataset['y_test_fine']],
                                           [dataset['y_test_coarse'],dataset['y_test_medium'],dataset['y_test_fine']]),
                        callbacks = [tb,log,change_lw,lr_decay,checkpoint],
                        verbose=1)
    model.save_weights(model_save_dir)

Epoch 1/100
780/781 [============================>.] - ETA: 0s - loss: 2.6292 - prediction_coarse_accuracy: 0.2532 - prediction_medium_accuracy: 0.5352 - prediction_fine_accuracy: 0.4527
  • Coarse Accuracy = 25.31% | Val_Accuracy = 14.32% | LossWeight = 0.49 
  • Medium Accuracy = 53.52% | Val_Accuracy = 66.26% | LossWeight = 0.22 
  • Fine   Accuracy = 45.28% | Val_Accuracy = 57.97% | LossWeight = 0.19 

Epoch 1: val_prediction_fine_accuracy improved from -inf to 0.57970, saving model to ../../logs/3_CIFAR_10/HD_CapsNet/Mod_4_1_2\epoch-best.h5
781/781 [==============================] - 48s 51ms/step - loss: 2.6288 - prediction_coarse_accuracy: 0.2531 - prediction_medium_accuracy: 0.5352 - prediction_fine_accuracy: 0.4528 - val_loss: 1.1481 - val_prediction_coarse_accuracy: 0.1432 - val_prediction_medium_accuracy: 0.6626 - val_prediction_fine_accuracy: 0.5797 - lr: 0.0010
Epoch 2/100
780/781 [============================>.] - ETA: 0s - loss: 1.7026 - prediction_coarse_accuracy: 0.1308

781/781 [==============================] - 35s 45ms/step - loss: -74.3075 - prediction_coarse_accuracy: 0.4024 - prediction_medium_accuracy: 0.4820 - prediction_fine_accuracy: 0.3849 - val_loss: -41.0269 - val_prediction_coarse_accuracy: 0.4000 - val_prediction_medium_accuracy: 0.5130 - val_prediction_fine_accuracy: 0.4363 - lr: 0.0010
Epoch 11/100
780/781 [============================>.] - ETA: 0s - loss: -74.4064 - prediction_coarse_accuracy: 0.4014 - prediction_medium_accuracy: 0.4870 - prediction_fine_accuracy: 0.3921
  • Coarse Accuracy = 40.12% | Val_Accuracy = 40.00% | LossWeight = 0.42 
  • Medium Accuracy = 48.71% | Val_Accuracy = 53.72% | LossWeight = 0.25 
  • Fine   Accuracy = 39.21% | Val_Accuracy = 44.41% | LossWeight = 0.23 

Epoch 11: val_prediction_fine_accuracy did not improve from 0.73160
781/781 [==============================] - 36s 46ms/step - loss: -74.4356 - prediction_coarse_accuracy: 0.4012 - prediction_medium_accuracy: 0.4871 - prediction_fine_accuracy: 0.392

781/781 [==============================] - 33s 42ms/step - loss: -75.5253 - prediction_coarse_accuracy: 0.3994 - prediction_medium_accuracy: 0.5988 - prediction_fine_accuracy: 0.5079 - val_loss: -41.6344 - val_prediction_coarse_accuracy: 0.4000 - val_prediction_medium_accuracy: 0.5885 - val_prediction_fine_accuracy: 0.5035 - lr: 3.7735e-04
Epoch 30/100
781/781 [==============================] - ETA: 0s - loss: -75.0141 - prediction_coarse_accuracy: 0.4027 - prediction_medium_accuracy: 0.5986 - prediction_fine_accuracy: 0.5084
  • Coarse Accuracy = 40.27% | Val_Accuracy = 40.00% | LossWeight = 0.47 
  • Medium Accuracy = 59.86% | Val_Accuracy = 62.29% | LossWeight = 0.22 
  • Fine   Accuracy = 50.84% | Val_Accuracy = 54.50% | LossWeight = 0.21 

Epoch 30: val_prediction_fine_accuracy did not improve from 0.73160
781/781 [==============================] - 33s 42ms/step - loss: -75.0141 - prediction_coarse_accuracy: 0.4027 - prediction_medium_accuracy: 0.5986 - prediction_fine_accuracy: 0

781/781 [==============================] - 36s 46ms/step - loss: -77.1033 - prediction_coarse_accuracy: 0.3962 - prediction_medium_accuracy: 0.6628 - prediction_fine_accuracy: 0.5850 - val_loss: -42.0958 - val_prediction_coarse_accuracy: 0.3997 - val_prediction_medium_accuracy: 0.6969 - val_prediction_fine_accuracy: 0.6207 - lr: 1.4240e-04
Epoch 49/100
781/781 [==============================] - ETA: 0s - loss: -76.1835 - prediction_coarse_accuracy: 0.4026 - prediction_medium_accuracy: 0.6748 - prediction_fine_accuracy: 0.5987
  • Coarse Accuracy = 40.26% | Val_Accuracy = 39.93% | LossWeight = 0.52 
  • Medium Accuracy = 67.48% | Val_Accuracy = 70.26% | LossWeight = 0.20 
  • Fine   Accuracy = 59.87% | Val_Accuracy = 62.62% | LossWeight = 0.18 

Epoch 49: val_prediction_fine_accuracy did not improve from 0.73160
781/781 [==============================] - 37s 48ms/step - loss: -76.1835 - prediction_coarse_accuracy: 0.4026 - prediction_medium_accuracy: 0.6748 - prediction_fine_accuracy: 0

KeyboardInterrupt: 

In [ ]:
try:
    history_dict = history.history

    plotter = tfdocs.plots.HistoryPlotter()
    plotter.plot({"Coarse": history}, metric = "prediction_coarse_accuracy")
    plotter.plot({"Medium": history}, metric = "prediction_medium_accuracy")
    plotter.plot({"Fine": history}, metric = "prediction_fine_accuracy")
    plt.title("Model Accuracy")
    plt.ylim([0,1])

except:
    print('Trained model weights loaded')

In [ ]:
try:
    plotter = tfdocs.plots.HistoryPlotter()
    plotter.plot({"loss": history}, metric = "loss")
    plt.title("Model Loss")
    plt.ylim([0,1])

except:
    print('Trained model weights loaded')

## Model Analysis

In [ ]:
final_model = keras.Model(model.inputs[:1], model.output)

lossfn = models.MarginLoss()
final_model.compile(optimizer='adam', 
                    loss={'prediction_fine' : lossfn,
                          'prediction_medium' : lossfn,
                          'prediction_coarse' : lossfn},

                    loss_weights={'prediction_fine' : lossweight['fine_lw'],
                                  'prediction_medium' : lossweight['medium_lw'],
                                  'prediction_coarse' : lossweight['coarse_lw']},

                    metrics={'prediction_fine': 'accuracy',
                             'prediction_medium': 'accuracy',
                             'prediction_coarse': 'accuracy'
                            }
                   )

In [ ]:
model_analysis = models.model_analysis(final_model, dataset)
results = model_analysis.evaluate()
predictions = model_analysis.prediction()

In [ ]:
true_label = [dataset['y_test_coarse'],dataset['y_test_medium'],dataset['y_test_fine']]
pred_label = [predictions[0],predictions[1],predictions[2]]
metrics.lvl_wise_metric(true_label,pred_label)

In [ ]:
h_measurements,consistency,exact_match = metrics.hmeasurements(true_label,
                                       pred_label,
                                       dataset['tree'])
print('\nHierarchical Precision =',h_measurements[0],
      '\nHierarchical Recall =', h_measurements[1],
      '\nHierarchical F1-Score =',h_measurements[2],
      '\nConsistency = ', consistency,
      '\nExact Match = ', exact_match,
     )